# Лабораторная работа №3. Методы поиска подстроки в строке.
## Выполнил студент группы БСТ1901 Мархель Сергей.


### Задание 1
### Алгоритм Кнута-Морриса-Пратта

In [49]:
def prefix(s):
    v = [0]*len(s)
    for i in range(1,len(s)):
        k = v[i-1]
        while k > 0 and s[k] != s[i]:
            k = v[k-1]
        if s[k] == s[i]:
            k = k + 1
        v[i] = k
    return v

def kmp(s, t):
    index = -1
    f = prefix(s)
    k = 0
    for i in range(len(t)):
        while k > 0 and s[k] != t[i]:
            k = f[k-1]
        if s[k] == t[i]:
            k = k + 1
        if k == len(s):
            index = i - len(s) + 1
            break
    return index

### Упрощенный Бойера-Мура

In [84]:
def bmPredCompil(x):
    d = {}
    lenX = len(x)
    for i in range(len(x)):
        # сколько символов с правого края до этой буквы
        d[ord(x[i])] = lenX - i
    return d

def boyerMurSearch(x, s):
    d = bmPredCompil(x)
    # k - проход по s
    # j - проход по x
    # i - место начала прохода по s
    lenX = i = j = k = len(x)
    while j > 0 and i<=len(s):
     # совпали, двигаемся дальше (от конца к началу)
        if s[k-1] == x[j-1]:
            k -= 1
            j -= 1
        # иначе, продвигаемся по строке на d и начинаем с правого конца подстроки снова
        else:
            i += d[ord(s[i])]
            j = lenX
            k = i
    if j <= 0:# нашли
        return k
    return None # не нашли

def search(s, t, alg=kmp, ignore_case=False, ignore_space=False):
    if ignore_case:
        s = s.lower()
        t = t.lower()
        
    if ignore_space:
        s = s.replace(' ', '')
        t = t.replace(' ', '')
        
    return alg(s, t)

3

In [82]:
import time
import string

alg = {
    'Кнута-Морриса-Пратта': kmp,
    'Бойера-Мура': boyerMurSearch,
      }

for key, a in alg.items():
    time_start = time.perf_counter()
    search('acD', 'aaaacd', alg=a, ignore_case=True, ignore_space=True)
    print(f'{key} - {time.perf_counter() - time_start}')
    
time_start = time.perf_counter()
'aaaacd'.find('acD')
print(f'Встроенная сортировка - {time.perf_counter() - time_start}')

Кнута-Морриса-Пратта - 1.4400000054592965e-05
Бойера-Мура - 1.4299999747890979e-05
Встроенная сортировка - 7.719999985056347e-05


### Задание 2

In [1]:
from queue import PriorityQueue

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))


def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

if parity(start) == 0:
    # https://en.wikipedia.org/wiki/15_puzzle#Solvability
    print('Нерешаемо')
else:
    start = tuple(start)

    p = Position(start, 0)

    print(p)
    print()

    candidates = PriorityQueue()
    candidates.put(p)

    visited = set([p])

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    path = []
    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]
